In [25]:
import pathlib
import fire as fire
from B_VAE.VAE_meta import b_encodeco
from Train_utils import train_utils
from B_VAE.Utils_imp_VAE import loss_fn
from Train_utils.train_utils import train,test,K_fold_train

import sys
import os
sys.path.append(os.path.join("..","Data_prep"))
from torchvision import transforms
from Custom_dataloader import *
from Transforms import phantom_segmentation
from Transforms import multi_image_resize
from Transforms import multi_ToTensor
from Transforms import output_transform
from Transforms import pos_fly_transform
from Transforms import concatenate_non_uniform_transform
from Transforms import only_tensor_transform
from Transforms import rgb_normalize
from Transforms import hue_transform

import torch
from torch import nn

import matplotlib.pyplot as plt

In [34]:
DB="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=aorus_1,volume=Paltas_DataBase/Data_Base_v2"
DB="//MYCLOUDPR4100/Paltas_DataBase/Data_Base_v2"

d_tt=transforms.Compose([
        phantom_segmentation(False,non_uniform_input=True),
        hue_transform(),
        #rgb_normalize(ImType=['PhantomRGB']),
        multi_image_resize(ImType=['PhantomRGB'],size=(300,300)),
        #pos_fly_transform(),
        #concatenate_non_uniform_transform(),
        multi_ToTensor(ImType=['PhantomRGB']),
        only_tensor_transform(),
        #output_transform()
        ])

In [35]:
datab=Dataset_direct(root_dir=DB,ImType=['PhantomRGB'],Intersec=False,transform=d_tt)

In [36]:
pr=datab[0]

In [39]:
#print(np.max(pr["PhantomRGB"]))
pr

{'PhantomRGB': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]])}

In [7]:
model=b_encodeco(
                 image_dim=int(20),
                 image_channels=3,
                 non_uniform_dim=32,
                 repr_sizes=[2,4],
                 pre_layer_sizes=[500],
                 layer_sizes=[300],
                 pre_output=400,
                 latent_space_size=20,
                 conv_kernel_size=5,
                 activators=[nn.Tanh(),nn.ReLU()],
                 conv_pooling=False,
                 conv_batch_norm=True,
                 NN_batch_norm=True,
                 stride=1,
                device='cpu')

In [8]:
model.eval()

b_encodeco(
  (encoder_conv): b_encoder_conv(
    (im_layers): ModuleList(
      (0): set_conv(
        (comp_layer): ModuleList(
          (0): Conv2d(3, 2, kernel_size=(5, 5), stride=(1, 1))
          (1): Tanh()
          (2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): set_conv(
        (comp_layer): ModuleList(
          (0): Conv2d(2, 4, kernel_size=(5, 5), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (pre_encoder): NeuralNet(
    (layers): ModuleList(
      (0): Sequential(
        (0): NN_layer(
          (layer): ModuleList(
            (0): Linear(in_features=608, out_features=500, bias=True)
            (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
        )
      )
      (1): Sequential(
        (0): NN_layer(
          (l

In [9]:
dataloader_train=torch.utils.data.DataLoader(datab,batch_size=1,shuffle=False,num_workers=0)

In [10]:
for batch in dataloader_train:
    #print(idx)
    break

In [11]:
batch

{'PhantomRGB': tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [1.0513e-03, 2.9332e-02, 1.4440e-01,  ..., 4.7411e-05,
            3.5064e-05, 2.2639e-06],
           [9.9853e-06, 4.9352e-03, 9.8949e-02,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [1.9663e-07, 1.0973e-03, 2.5193e-02,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00]],
 
          [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [1.0296e-03, 

In [12]:
z1,z2,mu,sig=model.forward_non_uniform(batch['PhantomRGB'],batch['Non_uniform_input'])

In [16]:
z1.shape

torch.Size([1, 3, 20, 20])

In [1]:
a=torch.from_numpy(np.random.uniform(0,4,(1,3)))
b=torch.from_numpy(np.random.uniform(0,4,(1,5)))
torch.concat((a,b),dim=1)


NameError: name 'torch' is not defined

In [ ]:
x1=batch['PhantomRGB']
x2=batch['Non_uniform_input']

x1=model.encoder_conv(x1)
x1=model.flatten(x1)
#Pre estimation
print(torch.concat((x1,x2.squeeze(1).squeeze(1)),dim=1)) #CHECK
x=model.pre_encoder(torch.concat((x1,x2.squeeze(1).squeeze(1)),dim=1))
#FCNN
mu=model.encoder_NN_mu(x)
sig=model.encoder_NN_sig(x)

z12=model.reparametrization(mu,sig)
#Pre estimation inv

z12=model.decoder_NN(z12)
z12=model.post_encoder(z12) #TODO: split image and other input latent variables
z1,z2=torch.split(z12,[model.NN_input,model.non_uniform_input_dim],dim=1)

z1=model.flatten(z1)
z1=model.decoder_conv(z1)

In [ ]:
torch.concat((x1,x2.squeeze(1).squeeze(1)),dim=1).shape

In [ ]:
x1.shape

In [ ]:
xx=x2.squeeze(1).squeeze(1)
xx.shape

In [ ]:
z1,z2=torch.split(z12,[model.NN_input,model.non_uniform_input_dim],dim=1)

In [ ]:
z2